In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats, leaguedashteamclutch, leaguegamelog, boxscoreplayertrackv2, playercareerstats, teamplayerdashboard, leaguestandings
from nba_api.stats.static import players, teams
import time
pd.set_option('display.max_columns', None)
from IPython import display as ICD
import warnings
from datetime import date
import pickle
warnings.filterwarnings('ignore')

In [2]:
abbrv_mapping = {'DEN':"1610612743", 
           'SAS':"1610612759", 
           'IND':"1610612754", 
           'MEM':"1610612763", 
           'NOP':"1610612740", 
           'CHI':"1610612741", 
           'MIN':"1610612750", 
           'MIA':"1610612748", 
           'ATL':"1610612737",
           'ORL':"1610612753", 
           'SAC':"1610612758", 
           'UTA':"1610612762", 
           'CHA':"1610612766", 
           'WAS':"1610612764", 
           'HOU':"1610612745", 
           'PHX':"1610612756", 
           'OKC':"1610612760", 
           'POR':"1610612757",
           'LAC':"1610612746", 
           'GSW':"1610612744", 
           'PHI':"1610612755", 
           'LAL':"1610612747", 
           'BKN':"1610612751", 
           'TOR':"1610612761", 
           'BOS':"1610612738", 
           'MIL':"1610612749", 
           'DET':"1610612765",
           'CLE':"1610612739", 
           'NYK':"1610612752",
           'DAL':"1610612742"}

name_mapping = {'Nuggets':"1610612743", 
           'Spurs':"1610612759", 
           'Pacers':"1610612754", 
           'Memphis':"1610612763", 
           'Pelicans':"1610612740", 
           'Bulls':"1610612741", 
           'Timberwolves':"1610612750", 
           'Heat':"1610612748", 
           'Hawks':"1610612737",
           'Magic':"1610612753", 
           'Kings':"1610612758", 
           'Jazz':"1610612762", 
           'Hornets':"1610612766", 
           'Wizards':"1610612764", 
           'Rockets':"1610612745", 
           'Suns':"1610612756", 
           'Thunder':"1610612760", 
           'Trail Blazers':"1610612757",
           'Clippers':"1610612746", 
           'Warriors':"1610612744", 
           '76ers':"1610612755", 
           'Lakers':"1610612747", 
           'Nets':"1610612751", 
           'Raptors':"1610612761", 
           'Celtics':"1610612738", 
           'Bucks':"1610612749", 
           'Pistons':"1610612765",
           'Cavaliers':"1610612739", 
           'Knicks':"1610612752",
           'Mavericks':"1610612742"}

In [3]:
def clean_all_playoff_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season_type_all_star='Playoffs', season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

def clean_all_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

all_playoff_games = clean_all_playoff_games('2019-20')
all_regseason_playoff_games = clean_all_games('2019-20')
all_reg_season_playoff_games = pd.concat([all_playoff_games, all_regseason_playoff_games]).sort_values('GameDate',ascending = False)
all_reg_season_playoff_games.head(20)

,GameID,GameDate,TeamID,OpponentTeamID,MATCHUP,PTS,OPTS
47,0041900143,2020-08-22,1610612747,1610612757,LAL @ POR,116,108
45,0041900173,2020-08-22,1610612745,1610612760,HOU @ OKC,107,119
44,0041900173,2020-08-22,1610612760,1610612745,OKC vs. HOU,119,107
43,0041900133,2020-08-22,1610612748,1610612754,MIA vs. IND,124,115
42,0041900133,2020-08-22,1610612754,1610612748,IND @ MIA,115,124
41,0041900103,2020-08-22,1610612749,1610612753,MIL @ ORL,121,107
40,0041900103,2020-08-22,1610612753,1610612749,ORL vs. MIL,107,121
46,0041900143,2020-08-22,1610612757,1610612747,POR vs. LAL,108,116
37,0041900123,2020-08-21,1610612755,1610612738,PHI vs. BOS,94,102
39,0041900153,2020-08-21,1610612746,1610612742,LAC @ DAL,130,122


In [4]:
player_rotation = pd.read_csv('players_rotation.csv')
players_df = pd.read_csv('players.csv')
first_game_missing_mins = player_rotation.groupby('TEAM_ID').sum()['MIN'] / players_df.groupby('TEAM_ID').sum()['MIN']

In [5]:
def missing_mins(curr_game_id, team_id, season_name):
    '''
    missing_mins: gets the fraction of the season's minutes played by injured/inactive/traded players
    '''
    time.sleep(0.5)
    # get the season stats of the team
    season_stats = teamplayerdashboard.TeamPlayerDashboard(team_id, season=season_name).get_data_frames()[1]
    
    # get the game currently being assessed
    curr_game = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=curr_game_id).get_data_frames()[0]
    
    player_game_stats = curr_game[curr_game['TEAM_ID'] == int(team_id)]
    combined = pd.merge(season_stats, player_game_stats, how='left', on='PLAYER_ID')
    
    total_mins = combined['MIN_x'].sum()
    missing_mins = combined[(combined['MIN_y'] == "0:00") | (combined['MIN_y'].isna())]['MIN_x'].sum()
    return (total_mins - missing_mins) / total_mins

In [6]:
games = pd.read_csv('predictions/games.csv')
games['Date'] = games['Date'].astype('datetime64')
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,41900161.0,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,41900111.0,1.610613e+09,1.610613e+09,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,41900121.0,1.610613e+09,1.610613e+09,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,41900151.0,1.610613e+09,1.610613e+09,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,41900101.0,1.610613e+09,1.610613e+09,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,41900131.0,1.610613e+09,1.610613e+09,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,41900171.0,1.610613e+09,1.610613e+09,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,41900141.0,1.610613e+09,1.610613e+09,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,41900162.0,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,41900112.0,1.610613e+09,1.610613e+09,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [7]:
def get_gameid(row):
    if(not pd.isna(row['GameID'])):
        return row['GameID']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = all_playoff_games[all_playoff_games['OpponentTeamID'] == row['OpponentTeamID']]
    temp = temp[temp['GameDate'].apply(lambda x: x.date() == row['Date'].date())]
    if(len(temp) != 0):
        return temp['GameID'].iloc[0]
    return row['GameID']
def get_game_info(row, attribute, alt_attribute):
    if(pd.isna(row['GameID'])):
        return row[alt_attribute]
    try:
        return all_playoff_games[(all_playoff_games['GameID'] == row['GameID']) & (all_playoff_games['TeamID'] == row['TeamID'])][attribute].iloc[0]
    except:
        return row[alt_attribute]
def get_game_num(row):
    if(not pd.isna(row['GameNum'])):
        return row['GameNum']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = all_playoff_games[all_playoff_games['OpponentTeamID'] == row['OpponentTeamID']]
    return len(temp) + 1

games['TeamID'] = games['TeamName'].apply(lambda x: int(name_mapping[x]))
games['OpponentTeamID'] = games['OpponentTeamName'].apply(lambda x: int(name_mapping[x]))
games['GameID'] = games.apply(lambda x: get_gameid(x), axis = 1)
games['Date'] = games.apply(lambda x: get_game_info(x, 'GameDate', 'Date'), axis = 1)
games['TeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'PTS', 'TeamPointsActual'), axis = 1)
games['OpponentTeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'OPTS', 'OpponentTeamPointsActual'), axis = 1)
games['GameNum'] = games.apply(lambda x: get_game_num(x), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [8]:
def get_series_lead(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series)
    games_won = len(series[series['PTS'] > series['OPTS']])
    return 2 * games_won - len_series

def get_series_win_percent(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series) + 1
    games_won = len(series[series['PTS'] > series['OPTS']])
    return games_won/len_series

In [9]:
# creating the games dataframe
games_copy = games[pd.isna(games['TeamWinProba'])][['Date','TeamID', 'OpponentTeamID']]
games_copy_reversed = games_copy.copy()
games_copy_reversed['TeamID'] = games_copy['OpponentTeamID']
games_copy_reversed['OpponentTeamID'] = games_copy['TeamID']

all_games = pd.concat([games_copy, games_copy_reversed]).sort_index().reset_index(drop=True)

reg_season_ratings = leaguestandings.LeagueStandings(season='2019-20', season_type = 'Regular Season').get_data_frames()[0]
reg_season_ratings = reg_season_ratings[['TeamID','PointsPG','OppPointsPG']]

#ratings
all_games = pd.merge(all_games, reg_season_ratings, how='left', on='TeamID')
all_games = pd.merge(all_games, reg_season_ratings, how='left', left_on='OpponentTeamID', right_on='TeamID', suffixes=['','Opponent'])

all_games['SeriesGP'] = all_games.apply(lambda x: 1 + len(all_playoff_games[(all_playoff_games['TeamID'] == x['OpponentTeamID']) & (all_playoff_games['OpponentTeamID'] == x['TeamID'])]), axis = 1) 
all_games['SeriesLead'] = all_games.apply(lambda x: get_series_lead(all_playoff_games, x), axis = 1) 
all_games['SeriesWinPercent'] = all_games.apply(lambda x: get_series_win_percent(all_playoff_games, x), axis = 1) 





all_games['PrevPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['PTS'].iloc[0], axis = 1) 
all_games['PrevOPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['OPTS'].iloc[0], axis = 1) 
all_games['PrevMargin'] = all_games['PrevPTS'] - all_games['PrevOPTS']


all_games['MissingMins'] = all_games.apply(lambda x: missing_mins(all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['GameID'].iloc[0], x['TeamID'], '2019-20'), axis = 1)
# first_game_missing_mins_copy = first_game_missing_mins.to_frame().reset_index()
# first_game_missing_mins_copy.columns = ['TeamID', 'MissingMins']
# all_games = pd.merge(all_games, first_game_missing_mins_copy, how='left', on='TeamID')


playoff_experience = pd.read_csv('playoff_experience.csv')
all_games = pd.merge(all_games, playoff_experience, how = 'left', on = 'TeamID')
all_games


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins
0,NaT,1610612743,1610612762,111.3,109.2,1610612762,111.3,108.8,4,-1,0.25,87,124,-37,0.718108,391.0,6895.763333
1,NaT,1610612762,1610612743,111.3,108.8,1610612743,111.3,109.2,4,1,0.50,124,87,37,0.819471,396.0,6328.350000
2,NaT,1610612761,1610612751,112.8,106.5,1610612751,111.8,112.3,4,3,0.75,117,92,25,0.873192,397.0,13204.620000
3,NaT,1610612751,1610612761,111.8,112.3,1610612761,112.8,106.5,4,-3,0.00,92,117,-25,0.446036,404.0,7254.545000
4,NaT,1610612738,1610612755,113.7,107.3,1610612755,110.7,108.4,4,3,0.75,102,94,8,0.814841,379.0,5772.186667
5,NaT,1610612755,1610612738,110.7,108.4,1610612738,113.7,107.3,4,-3,0.00,94,102,-8,0.734108,403.0,7951.311667
6,NaT,1610612746,1610612742,116.3,109.9,1610612742,117.0,112.1,4,1,0.50,130,122,8,0.713266,412.0,13322.215000
7,NaT,1610612742,1610612746,117.0,112.1,1610612746,116.3,109.9,4,-1,0.25,122,130,-8,0.761565,415.0,3609.745000
8,NaT,1610612749,1610612753,118.7,108.6,1610612753,107.3,108.3,4,1,0.50,121,107,14,0.960203,446.0,16320.056667
9,NaT,1610612753,1610612749,107.3,108.3,1610612749,118.7,108.6,4,-1,0.25,107,121,-14,0.696825,393.0,4178.705000


In [10]:
# bias boosted monte carlo simulation

In [11]:
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
def get_weight(x, col_value, range_val):
    return 1/(00.1+abs((x-col_value)/range_val))
def get_rand_sample(df, col_name, col_value, sample_col_name, N, range_val):
    weights = df[col_name].apply(lambda x: get_weight(x,col_value, range_val))
    to_return = random.choices(population=list(df[sample_col_name]), 
                               weights=weights,
                               k=N)
    return to_return

In [12]:
pts_model = pickle.load( open( "pts_model", "rb" ) )
opts_model = pickle.load( open( "opts_model", "rb" ) )
range_vals = pickle.load( open( "range_vals", "rb" ) )
playoff_data = pd.read_csv('all_playoff_games.csv')
offense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','PointsPG','OppPointsPGOpponent','PrevPTS']
defense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','OppPointsPG','PointsPGOpponent','PrevOPTS']

In [13]:
def get_predictions(playoff_games, cols, predictor, N, model = None):
    predictions = playoff_games[['Date', 'TeamID','OpponentTeamID']]
    col_sum = []
    for x in cols:
        col_sum.append(x + "Sum")
    for col in cols:
        predictions[col] = playoff_games.apply(lambda x: get_rand_sample(playoff_data,col,x[col],predictor,N,range_vals[col]), axis = 1)
    for col in range(len(cols)):
        predictions[col_sum[col]] = predictions[cols[col]].apply(lambda x: sum(x)/N)
    X = predictions[col_sum]
    
    coefs = pd.Series(model.coef_, index = col_sum).abs()
    # predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: (np.array(x.values.tolist()) * coefs.values[:, None]).sum(axis=0)/coefs.sum(), axis = 1)
    predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: model.predict(np.array(x.values.tolist()).transpose()), axis = 1)
    predictions[predictor + 'LinearRegression'] = model.predict(X)
    temp = predictions[['TeamID'] + col_sum + [predictor+'LinearRegression']]
    temp['Name'] = temp['TeamID'].apply(lambda x: list(name_mapping.keys())[list(pd.Series(list(name_mapping.values())).astype(int)).index(x)])
    ICD.display(temp)
    temp.to_csv('predictions_intermed/' + predictor+date.today().strftime('%d-%m-%Y') + '.csv', index = False)
    return predictions[[predictor + 'Combined', predictor + 'LinearRegression']]

In [14]:
pts_df = get_predictions(all_games, offense_cols, 'AdjustedPTS', 10000, pts_model)
opts_df = get_predictions(all_games, defense_cols, 'AdjustedOPTS', 10000, opts_model)
final = pd.concat([all_games, pts_df, opts_df], axis = 1)
final['WinProba'] = final.apply(lambda x: np.count_nonzero(x['AdjustedPTSCombined'] > x['AdjustedOPTSCombined'])/1000, axis = 1)
final['WinProbaScaled'] = final.apply(lambda x: x['WinProba']* 1/final[((final['SeriesGP'] == x['SeriesGP'])) & (((final['TeamID'] == x['TeamID']) & (final['OpponentTeamID'] == x['OpponentTeamID'])) | ((final['TeamID'] == x['OpponentTeamID']) & (final['OpponentTeamID'] == x['TeamID'])))]['WinProba'].sum(), axis = 1)
final['WinProba2x'] = final.apply(lambda x: max(0,min(1,2*x['WinProbaScaled']-0.5)), axis = 1)
ICD.display(final)
final = final[['Date','TeamID','OpponentTeamID','AdjustedPTSLinearRegression','AdjustedOPTSLinearRegression', 'WinProba2x']]

,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,PointsPGSum,OppPointsPGOpponentSum,PrevPTSSum,AdjustedPTSLinearRegression,Name
0,1610612743,105.321824,106.085427,106.182597,105.371058,105.329503,105.400713,105.479525,106.331283,105.823125,104.620098,100.517106,Nuggets
1,1610612762,105.228105,106.024791,106.465626,106.479202,106.232437,106.210202,106.395820,106.338350,106.036007,107.581486,108.947377,Jazz
2,1610612761,105.270533,106.092635,106.486936,107.223456,107.270544,106.411862,106.458612,107.087751,107.116465,107.570024,113.559676,Raptors
3,1610612751,105.192277,106.283421,106.313249,105.815445,105.977901,105.584089,105.020790,106.537852,106.017263,104.480790,101.371141,Nets
4,1610612738,105.316347,106.172085,106.397869,107.345721,107.384980,106.282690,106.444218,107.402825,105.889714,105.618271,108.608295,Celtics
5,1610612755,105.211285,106.117075,106.226954,105.764471,105.856904,105.534099,105.470880,106.420756,105.875535,104.644312,101.427003,76ers
6,1610612746,105.365362,105.914823,106.299074,106.368031,106.093418,105.523202,106.326733,108.460376,107.127328,107.207944,112.185976,Clippers
7,1610612742,105.519945,106.010535,106.350421,105.391906,105.474549,105.660647,105.536234,108.298934,106.291707,107.683533,109.758587,Mavericks
8,1610612749,105.942482,106.193788,106.479303,106.451787,106.239042,106.757787,106.871069,108.498199,106.202208,107.581803,114.621797,Bucks
9,1610612753,105.287644,106.070489,106.358149,105.172339,105.615323,105.442246,105.657589,104.486987,105.932033,106.157029,101.198869,Magic


,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,OppPointsPGSum,PointsPGOpponentSum,PrevOPTSSum,AdjustedOPTSLinearRegression,Name
0,1610612743,106.359715,105.736897,106.253667,106.313034,106.218039,105.794264,106.811392,106.165466,106.385634,107.467089,109.414006,Nuggets
1,1610612762,106.423163,106.150567,106.159329,105.284551,104.967895,105.473247,105.561114,106.069626,106.592254,104.748208,103.622370,Jazz
2,1610612761,105.848317,106.203342,106.562414,105.771910,105.731624,106.287173,105.179649,106.022982,106.422369,104.920755,105.835316,Raptors
3,1610612751,106.379777,106.169730,106.267518,107.486530,107.077507,105.868731,106.929993,107.253596,106.893367,107.557560,114.481109,Nets
4,1610612738,106.740782,106.010415,106.237983,105.817853,105.727601,105.479918,105.738733,106.286587,106.338946,104.927006,105.231001,Celtics
5,1610612755,106.278241,105.963851,106.409809,107.396407,107.197533,106.087958,106.235324,105.989403,107.283410,105.550681,110.042327,76ers
6,1610612746,105.637546,106.200426,106.231978,104.929748,104.954160,105.785069,105.487760,106.240195,108.427623,107.641094,110.381653,Clippers
7,1610612742,106.785004,106.077959,106.468633,106.586732,106.184086,105.757318,106.096866,107.213435,108.549788,107.406734,116.340729,Mavericks
8,1610612749,105.239456,105.860936,106.418848,105.340460,104.822695,106.631623,106.008967,105.926204,104.515322,106.057753,102.892979,Bucks
9,1610612753,106.885175,106.279628,106.550316,106.414017,106.163646,105.813989,106.642588,106.011092,108.379159,107.717746,114.495140,Magic


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins,AdjustedPTSCombined,AdjustedPTSLinearRegression,AdjustedOPTSCombined,AdjustedOPTSLinearRegression,WinProba,WinProbaScaled,WinProba2x
0,NaT,1610612743,1610612762,111.3,109.2,1610612762,111.3,108.8,4,-1,0.25,87,124,-37,0.718108,391.0,6895.763333,"[141.0786683295205, 148.60316274500724, 78.193...",100.517106,"[42.35421943668257, 148.4963942749198, 133.739...",109.414006,4.566,0.463977,0.427954
1,NaT,1610612762,1610612743,111.3,108.8,1610612743,111.3,109.2,4,1,0.50,124,87,37,0.819471,396.0,6328.350000,"[98.05481762516774, 114.03896991748297, 69.288...",108.947377,"[159.93144737440548, 17.600690970386268, 160.7...",103.622370,5.275,0.536023,0.572046
2,NaT,1610612761,1610612751,112.8,106.5,1610612751,111.8,112.3,4,3,0.75,117,92,25,0.873192,397.0,13204.620000,"[68.72900447927827, 117.93658973370816, 124.81...",113.559676,"[23.634165263843443, -21.803526143860836, 149....",105.835316,5.362,0.557438,0.614877
3,NaT,1610612751,1610612761,111.8,112.3,1610612761,112.8,106.5,4,-3,0.00,92,117,-25,0.446036,404.0,7254.545000,"[84.97144584033185, 70.01025869167393, 224.387...",101.371141,"[87.79609152383773, 155.64329613576592, 205.03...",114.481109,4.257,0.442562,0.385123
4,NaT,1610612738,1610612755,113.7,107.3,1610612755,110.7,108.4,4,3,0.75,102,94,8,0.814841,379.0,5772.186667,"[54.65508366806466, 124.65602585088368, 137.91...",108.608295,"[104.85489922645434, -11.005012838822722, 57.1...",105.231001,5.208,0.532134,0.564269
5,NaT,1610612755,1610612738,110.7,108.4,1610612738,113.7,107.3,4,-3,0.00,94,102,-8,0.734108,403.0,7951.311667,"[69.99036834167714, 141.31787501109147, 97.403...",101.427003,"[174.05516050038386, 73.40078859050277, 9.5240...",110.042327,4.579,0.467866,0.435731
6,NaT,1610612746,1610612742,116.3,109.9,1610612742,117.0,112.1,4,1,0.50,130,122,8,0.713266,412.0,13322.215000,"[23.019177066475322, 102.06875801676892, 130.4...",112.185976,"[160.10183187953362, 107.34115564205217, 93.93...",110.381653,5.145,0.523877,0.547755
7,NaT,1610612742,1610612746,117.0,112.1,1610612746,116.3,109.9,4,-1,0.25,122,130,-8,0.761565,415.0,3609.745000,"[77.54390588853516, 90.73624263607985, 70.4729...",109.758587,"[182.15260934163462, 87.95829273635331, 197.68...",116.340729,4.676,0.476123,0.452245
8,NaT,1610612749,1610612753,118.7,108.6,1610612753,107.3,108.3,4,1,0.50,121,107,14,0.960203,446.0,16320.056667,"[134.0102767062242, 112.83705731145415, 150.81...",114.621797,"[121.01963983270116, 134.74731541905294, 30.61...",102.892979,5.576,0.564944,0.629889
9,NaT,1610612753,1610612749,107.3,108.3,1610612749,118.7,108.6,4,-1,0.25,107,121,-14,0.696825,393.0,4178.705000,"[192.62149273459477, 190.75486417416255, 154.5...",101.198869,"[177.24061784991204, 19.92424069275421, 165.82...",114.495140,4.294,0.435056,0.370111


In [15]:
for x in range(1, len(final),2):
    temp = final['AdjustedPTSLinearRegression'].iloc[x]
    final['AdjustedPTSLinearRegression'].iloc[x] = final['AdjustedOPTSLinearRegression'].iloc[x]
    final['AdjustedOPTSLinearRegression'].iloc[x] = temp
final =final.groupby(np.arange(len(final))//2).agg({'Date': 'first',
                                            'TeamID':'first',
                                             'OpponentTeamID':'first',
                                             'AdjustedPTSLinearRegression':'mean',
                                             'AdjustedOPTSLinearRegression':'mean',
                                             'WinProba2x':'first'})

In [16]:
import scipy.stats
norm_dist = scipy.stats.norm(0, 10)
final['ProbaNorm'] = final.apply(lambda x: norm_dist.cdf(x['AdjustedPTSLinearRegression'] - x['AdjustedOPTSLinearRegression']), axis = 1)
final

,Date,TeamID,OpponentTeamID,AdjustedPTSLinearRegression,AdjustedOPTSLinearRegression,WinProba2x,ProbaNorm
0,NaT,1610612743,1610612762,102.069738,109.180692,0.427954,0.238513
1,NaT,1610612761,1610612751,114.020392,103.603228,0.614877,0.851228
2,NaT,1610612738,1610612755,109.325311,103.329002,0.564269,0.725624
3,NaT,1610612746,1610612742,114.263353,110.070120,0.547755,0.662510
4,NaT,1610612749,1610612753,114.558468,102.045924,0.629889,0.894579
5,NaT,1610612754,1610612748,106.266315,110.170980,0.458790,0.348096
6,NaT,1610612745,1610612760,110.128723,110.059857,0.500602,0.502747
7,NaT,1610612747,1610612757,113.524780,106.421860,0.577809,0.761238


In [17]:
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [18]:
def get_game_info_calculated(row, final_col, games_col):
    if(not pd.isna(row[games_col])):
        return row[games_col]
    temp = final[final['TeamID'] == row['TeamID']]
    temp = final[final['OpponentTeamID'] == row['OpponentTeamID']]
    return temp[final_col].iloc[0]

games['TeamWinProba'] = games.apply(lambda x: get_game_info_calculated(x, 'ProbaNorm', 'TeamWinProba'), axis = 1)
games['OpponentTeamWinProba'] = games.apply(lambda x: 1 - x['TeamWinProba'], axis = 1)
games['TeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedPTSLinearRegression', 'TeamPoints'), axis = 1)
games['OpponentTeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedOPTSLinearRegression', 'OpponentTeamPoints'), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [20]:
today = date.today()

pd.read_csv('predictions/games.csv').to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_before.csv', index = False)
games.to_csv('predictions/games.csv', index = False)
games.to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_after.csv', index = False)